# LAB 3
### prepared by Markov Artur

In [1]:
import pandas as pd
import numpy as np

## Task 3. Tidying data



### Data

Dataset `hike_data.rds` contains information about hiking routes from (Washington Trail Association)\[https://www.wta.org/go-outside/hikes?b_start:int=1\].

Data contains the next columns:

| variable    | class     | description                                            |
|:------------------------|:---------------------|:------------------------|
| name        | character | Name of trail                                          |
| location    | character | Location of Trail                                      |
| length      | character | Length of trail (note that most have `miles` included) |
| gain        | character | Gain in elevation (Feet above sea level)               |
| highpoint   | character | Highest point in feet above sea level                  |
| rating      | character | User submitted rating (out of 5)                       |
| features    | character | Features                                               |
| description | character | Description of trail                                   |


### Data transforming

Because of data formatting in `hike_data.rds` -- none of the packages (`r2py`, `pyreadr` etc) were unable to transform .rds to .csv
So I come up with solution in R:
The code listed below
``` r
folder_from <- '/Users/arturmarkov/univer/master_degree/software_for_data_processing/L3/hike_data.rds'
folder_to <- '/Users/arturmarkov/univer/master_degree/software_for_data_processing/L3/hike_data.csv'
df_with_special_characters <- readRDS(folder_from)


df_with_special_characters['features'] = apply(df_with_special_characters['features'], 1:2, FUN=function(x) gsub("  ", "", paste(unlist(x, use.names = FALSE), collapse = ', ')))
head(df_with_special_characters, 1)
write.csv(df_with_special_characters, folder_to, row.names=FALSE)

```

In [2]:
df = pd.read_csv('/Users/arturmarkov/univer/master_degree/software_for_data_processing/L3/hike_data.csv')

In [3]:
df[:3]

,name,location,length,gain,highpoint,rating,features,description
0,Lake Hills Greenbelt,Puget Sound and Islands -- Seattle-Tacoma Area,"2.3 miles, roundtrip",50,330.0,3.67,"Dogs allowed on leash, Wildlife, Good for kids...",Hike through a pastoral area first settled and...
1,Snow Lake,Snoqualmie Region -- Snoqualmie Pass,"7.2 miles, roundtrip",1800,4400.0,4.16,"Ridges/passes, Dogs allowed on leash, Establis...",A relatively short and easy hike within a ston...
2,Skookum Flats,Mount Rainier Area -- Chinook Pass - Hwy 410,"7.8 miles, roundtrip",300,2550.0,3.68,"Wildflowers/Meadows, Mountain views, Wildlife,...",Choose between a shorter or longer river walk ...


### Tidying dataset

Create a new dataset `clean_hike_trails` with the next updates:


In [4]:
clean_hike_trails = df.copy()

1.  Convert columns `gain`, `highpoint`, `rating` to numeric values.

In [5]:

for col in ['gain', 'highpoint', 'rating']:
    clean_hike_trails[col] = clean_hike_trails[col].astype(float)


2.  Add new column `trip` with the type of trip from column `length` ("roundtrip", "trails", "one-way").

In [6]:
_bins = ["roundtrip", "trails", "one-way"]
_conditions = [
    clean_hike_trails.length.str.contains(item) for item in _bins
]

clean_hike_trails['trip'] = np.select(_conditions, _bins)

3.  Add new column `length_total` with the route length from column `length`, considering that for "one-way" trip you must double the route length.

In [7]:
# we will find all positive floats and even integers
clean_hike_trails['length_total'] = clean_hike_trails.length.fillna('').astype(str).str.extract('(\d+.\d+|\d+)',flags=0).astype(float)
clean_hike_trails.loc[clean_hike_trails.trip=='one-way', 'length_total'] *=2

4.  Add new column `location_general` with location from column `location` (a part before "--").

In [8]:
clean_hike_trails['location_general'] = clean_hike_trails.location.fillna('').astype(str).str.split('--').str[0].str.rstrip(' ')


5. Add column `id` with row number

In [9]:
clean_hike_trails = clean_hike_trails.reset_index().rename(columns={'index':'id'})

### Questioning dataset


**Question 1.** How many routes have rating more than 4.9


In [10]:
clean_hike_trails[:2]

,id,name,location,length,gain,highpoint,rating,features,description,trip,length_total,location_general
0,0,Lake Hills Greenbelt,Puget Sound and Islands -- Seattle-Tacoma Area,"2.3 miles, roundtrip",50.0,330.0,3.67,"Dogs allowed on leash, Wildlife, Good for kids...",Hike through a pastoral area first settled and...,roundtrip,2.3,Puget Sound and Islands
1,1,Snow Lake,Snoqualmie Region -- Snoqualmie Pass,"7.2 miles, roundtrip",1800.0,4400.0,4.16,"Ridges/passes, Dogs allowed on leash, Establis...",A relatively short and easy hike within a ston...,roundtrip,7.2,Snoqualmie Region


In [11]:
clean_hike_trails[clean_hike_trails.rating>4.9].id.count()

68


**Question 2.** How many routes are "Good for kids" (hint: you can use (`unnest` function)?


In [12]:
clean_hike_trails[clean_hike_trails.features.fillna('').str.contains('Good for kids')].shape[0]

694

**Question 3.** Which unique features can routes have?

In [13]:
sorted(pd.Series(np.hstack(clean_hike_trails.features.fillna('').astype(str).str.split(', ', expand=True).values)).dropna().unique())[1:]


['Coast',
 'Dogs allowed on leash',
 'Dogs not allowed',
 'Established campsites',
 'Fall foliage',
 'Good for kids',
 'Lakes',
 'Mountain views',
 'Old growth',
 'Ridges/passes',
 'Rivers',
 'Summits',
 'Waterfalls',
 'Wildflowers/Meadows',
 'Wildlife']


**Question 4.** What is the most common rating of a route?


In [14]:
clean_hike_trails.rating.value_counts().iloc[:1]

0.0    292
Name: rating, dtype: int64


**Question 5.** Your own question and answer.

Avarage `length_total` for each `loaction_general` by `trip` type

In [15]:
clean_hike_trails.pivot_table(columns='trip',index='location_general', values='length_total', aggfunc=pd.Series.mean)

trip,one-way,roundtrip,trails
location_general,,,
Central Cascades,28.366667,8.724872,4.000000
Central Washington,33.400000,5.526582,NaN
Eastern Washington,24.709091,8.798696,8.331667
Issaquah Alps,8.526000,4.642769,19.350000
Mount Rainier Area,33.000000,7.724731,NaN
North Cascades,46.881818,10.146344,NaN
Olympic Peninsula,30.050000,7.904643,6.080000
Puget Sound and Islands,18.440000,3.377643,7.827083
Snoqualmie Region,16.600000,8.762654,NaN
